<a href="https://colab.research.google.com/github/SantayogithubIT/WalkableArea/blob/main/PathDitection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics opencv-python pyttsx3


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import pyttsx3


In [ ]:
model = YOLO("yolov8n-seg.pt")



In [ ]:
!pip install gTTS
from gtts import gTTS
from IPython.display import Audio, display


In [ ]:
import os

def speak(text, filename="voice.mp3"):
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    display(Audio(filename, autoplay=True))


In [ ]:
from google.colab import files
import cv2
import numpy as np
from ultralytics import YOLO
from gtts import gTTS
from IPython.display import Audio, display
from google.colab.patches import cv2_imshow
import time

# ---------------------------
# TEXT-TO-SPEECH FUNCTION
# ---------------------------
def speak(text, filename="voice.mp3"):
    print("[AUDIO]:", text)
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    display(Audio(filename, autoplay=True))

# ---------------------------
# Load YOLO Segmentation Model
# ---------------------------
model = YOLO("yolov8n-seg.pt")

# Upload video
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

cap = cv2.VideoCapture(video_path)

# ---------------------------
# Helper function
# ---------------------------
def get_direction(center_x, width):
    left = width // 3
    right = 2 * width // 3

    if center_x < left:
        return "Move left"
    elif center_x > right:
        return "Move right"
    else:
        return "Move forward"

last_direction = ""
last_obstacle_time = 0
last_speak_time = 0

OBSTACLE_CLASSES = [0, 1, 2, 3]  # person, bicycle, car, motorcycle (adjust as needed)

# ---------------------------
# MAIN PROCESS LOOP
# ---------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_w = frame.shape[1]
    frame_h = frame.shape[0]

    results = model(frame, verbose=False)[0]

    walk_center_x = None
    mask_found = False

    # ---------------------------
    # MASK PROCESSING
    # ---------------------------
    if results.masks is not None:
        masks = results.masks.data.cpu().numpy()

        # Pick LARGEST mask
        areas = [np.sum(m) for m in masks]
        largest_idx = int(np.argmax(areas))

        mask = masks[largest_idx]
        mask = cv2.resize(mask, (frame_w, frame_h))

        mask_found = True

        # Get walkable center
        ys, xs = np.where(mask > 0.5)
        if len(xs) > 0:
            walk_center_x = int(np.mean(xs))

        # Overlay mask (green transparent)
        overlay = frame.copy()
        overlay[mask > 0.5] = (0, 255, 0)
        frame = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)

    # ---------------------------
    # OBSTACLE DETECTION
    # ---------------------------
    obstacle_detected = False

    for box in results.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])

        if cls in OBSTACLE_CLASSES and conf > 0.5:
            obstacle_detected = True

            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 2)
            cv2.putText(frame, "Obstacle", (x1, y1-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

    # Speak only once every 2 seconds
    if obstacle_detected and time.time() - last_obstacle_time > 2:
        speak("Obstacle ahead!")
        last_obstacle_time = time.time()

    # ---------------------------
    # DIRECTION GUIDANCE
    # ---------------------------
    if walk_center_x is not None:
        direction = get_direction(walk_center_x, frame_w)

        cv2.circle(frame, (walk_center_x, frame_h//2), 8, (255, 0, 0), -1)
        cv2.putText(frame, direction, (50,50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        # Speak only if direction changed
        if direction != last_direction and time.time() - last_speak_time > 1.5:
            speak(direction)
            last_direction = direction
            last_speak_time = time.time()

    else:
        cv2.putText(frame, "No walkable path detected", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # ---------------------------
    # SHOW OUTPUT FRAME
    # ---------------------------
    cv2_imshow(frame)

cap.release()


In [ ]:
from google.colab import files

uploaded = files.upload()
video_path = list(uploaded.keys())[0]

cap = cv2.VideoCapture(video_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    annotated = results[0].plot()    # YOLO segmentation overlay

    cv2_imshow(annotated)

cap.release()


In [ ]:
!pip install ultralytics gTTS

import cv2
import numpy as np
from ultralytics import YOLO
from gtts import gTTS
from IPython.display import Audio, display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# -------------------------------
# gTTS Speak Function
# -------------------------------
def speak(text, filename="direction.mp3"):
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    display(Audio(filename, autoplay=True))

# -------------------------------
# Webcam Capture (JavaScript)
# -------------------------------
def capture_frame():
    js_code = Javascript('''
        async function takePhoto() {
          const video = document.createElement('video');
          document.body.appendChild(video);
          const stream = await navigator.mediaDevices.getUserMedia({video: true});
          video.srcObject = stream;
          await video.play();

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          const context = canvas.getContext('2d');
          context.drawImage(video, 0, 0);

          stream.getTracks().forEach(t => t.stop());
          document.body.removeChild(video);

          return canvas.toDataURL('image/jpeg');
        }
        takePhoto();
    ''')

    display(js_code)
    data = eval_js('takePhoto()')
    img_bytes = b64decode(data.split(',')[1])

    with open("frame.jpg", "wb") as f:
        f.write(img_bytes)

    return cv2.imread("frame.jpg")

# -------------------------------
# Load YOLO Segmentation Model
# -------------------------------
model = YOLO("yolov8n-seg.pt")


last_direction = ""  # avoid repeated speaking

def get_direction(cx, width):
    if cx < width / 3:
        return "Move left"
    elif cx > 2 * width / 3:
        return "Move right"
    else:
        return "Move forward"

# -------------------------------
# MAIN LOOP (Capture → Detect → Speak)
# -------------------------------
while True:
    print("📸 Capturing frame... Look at your webcam.")
    frame = capture_frame()

    if frame is None:
        print("❌ No frame captured.")
        break

    results = model(frame)[0]

    walk_center_x = None

    if results.masks is not None:
        masks = results.masks.data.cpu().numpy()

        # Pick largest mask (walkable region)
        areas = [np.sum(mask) for mask in masks]
        idx = int(np.argmax(areas))

        mask = cv2.resize(masks[idx], (frame.shape[1], frame.shape[0]))

        ys, xs = np.where(mask > 0.5)
        if len(xs) > 0:
            walk_center_x = int(np.mean(xs))

    # -----------------------
    # Decision + Voice Output
    # -----------------------
    if walk_center_x is not None:
        frame_w = frame.shape[1]
        direction = get_direction(walk_center_x, frame_w)

        print("➡️ Direction:", direction)

        if direction != last_direction:
            speak(direction)
            last_direction = direction
    else:
        print("⚠️ No walkable area detected")
        speak("No walkable area detected")

    # -----------------------
    # Show annotated image
    # -----------------------
    annotated = results.plot()
    cv2_imshow(annotated)

    # Ask user if they want to continue
    cont = input("Capture another frame? (y/n): ")
    if cont.lower() != "y":
        break

print("✅ Finished.")

